In [ ]:
pip install transformers torch


RoBERTa Implementation

In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset

# Load the datasets
train_file_path = '/content/Tam-SA-train.csv'
test_file_path = '/content/Tam-SA-test-without-labels.csv'

train_data = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)

# Analyze class distribution
print(train_data['Label'].value_counts(normalize=True))

# Preprocessing function for text data
def preprocess_text(text):
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove special characters
    text = re.sub(r"\s+", " ", text).strip()    # Remove extra spaces
    return text.lower()

# Apply preprocessing
train_data['Cleaned_Text'] = train_data['Text'].apply(preprocess_text)
test_data['Cleaned_Text'] = test_data['Text'].apply(preprocess_text)

# Encode labels
label_encoder = LabelEncoder()
train_data['Encoded_Label'] = label_encoder.fit_transform(train_data['Label'])

# Define label mapping for reference
label_mapping = {index: label for index, label in enumerate(label_encoder.classes_)}
print("Label Mapping:", label_mapping)

# Split training data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    train_data['Cleaned_Text'], train_data['Encoded_Label'],
    test_size=0.2, random_state=42, stratify=train_data['Encoded_Label']
)

# Custom dataset class
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts.tolist()
        self.labels = labels.tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Load RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_mapping))

# Prepare datasets
train_dataset = SentimentDataset(X_train, y_train, tokenizer, max_length=128)
val_dataset = SentimentDataset(X_val, y_val, tokenizer, max_length=128)

# Training setup
training_args = TrainingArguments(
    output_dir="./roberta_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

# Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()

# Save the model and tokenizer
model.save_pretrained("./roberta_sentiment_model")
tokenizer.save_pretrained("./roberta_sentiment_model")

# Prepare test dataset
test_dataset = SentimentDataset(test_data['Cleaned_Text'], [0] * len(test_data), tokenizer, max_length=128)

# Predict on the test set
test_predictions = trainer.predict(test_dataset)

# Map predictions back to labels
predicted_labels = label_encoder.inverse_transform(torch.argmax(torch.tensor(test_predictions.predictions), axis=1))
test_data['Predicted_Label'] = predicted_labels

# Save predictions
test_data[['Text', 'Predicted_Label']].to_csv("Tam-SA-test-Robert-predictions.csv", index=False)

print("Test predictions saved to Tam-SA-test-predictions.csv")


Label
Positive          0.583028
unknown_state     0.165928
Negative          0.133378
Mixed_feelings    0.117666
Name: proportion, dtype: float64
Label Mapping: {0: 'Mixed_feelings', 1: 'Negative', 2: 'Positive', 3: 'unknown_state'}


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss


cnn


XLNet Implementation